In [53]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
import seaborn as sns
import matplotlib.pyplot as plt



In [7]:
true = pd.read_csv('/content/sample_data/Fakenews/True.csv')

fake = pd.read_csv('/content/sample_data/Fakenews/Fake.csv')


In [8]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [9]:
true['label'] = 1
fake['label'] = 0

In [10]:
fake.head()


,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [14]:
news = pd.concat([fake, true], axis = 0)
news.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [15]:
news.tail()

,title,text,subject,date,label
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [16]:
news.isnull().sum()

,0
title,0
text,0
subject,0
date,0
label,0


In [18]:
news = news.drop(['title', 'subject', 'date'], axis = 1)

In [19]:
## Reshuffle data, currently all labels with 1 value are below
news= news.sample(frac = 1)
news.head()

,text,label
3870,"Right now, it s looking increasingly like Hill...",0
14783,Obama is determined to close GITMO to follow t...,0
8061,Republican front runner Donald Trump loves to ...,0
7838,"As you may or may not know at this point, MSNB...",0
17659,When Keurig decided to side with Soros against...,0


In [20]:
news.reset_index(inplace = True)
news.head()

,index,text,label
0,3870,"Right now, it s looking increasingly like Hill...",0
1,14783,Obama is determined to close GITMO to follow t...,0
2,8061,Republican front runner Donald Trump loves to ...,0
3,7838,"As you may or may not know at this point, MSNB...",0
4,17659,When Keurig decided to side with Soros against...,0


In [21]:
news.drop(['index'], axis = 1, inplace = True)
news.head()

,text,label
0,"Right now, it s looking increasingly like Hill...",0
1,Obama is determined to close GITMO to follow t...,0
2,Republican front runner Donald Trump loves to ...,0
3,"As you may or may not know at this point, MSNB...",0
4,When Keurig decided to side with Soros against...,0


In [24]:
 def wordopt(text):

    text = text.lower()

  #To remove the links
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

  #To remove the html tags
    text = re.sub(r'<.*?>+', '', text)

    text =  re.sub(r'\n', ' ', text)
    return text

In [25]:
news['text'] = news ['text'].apply(wordopt)

In [26]:
news['text']

,text
0,"right now, it s looking increasingly like hill..."
1,obama is determined to close gitmo to follow t...
2,republican front runner donald trump loves to ...
3,"as you may or may not know at this point, msnb..."
4,when keurig decided to side with soros against...
...,...
44893,german social democrat leader martin schulz sa...
44894,"during an appearance on cnn on tuesday, republ..."
44895,tokyo (reuters) - a court in japan on friday o...
44896,berlin (reuters) - she has earned a reputation...


In [27]:
x = news['text']
y = news['label']

In [32]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)
#

In [33]:
x_train.shape

(31428,)

In [34]:
x_test.shape

(13470,)

In [36]:
### Text to vectro
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)
xv_test

<13470x102902 sparse matrix of type '<class 'numpy.float64'>'
	with 2793849 stored elements in Compressed Sparse Row format>

In [37]:
## train model
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [39]:
pred_lr = LR.predict(xv_test)
LR.score(xv_test, y_test)


0.9869339272457313

In [42]:
DTC = DecisionTreeClassifier()
DTC.fit(xv_train, y_train)

DecisionTreeClassifier()

In [43]:
pred_dt = DTC.predict(xv_test)
DTC.score(xv_test, y_test)

0.99547141796585

In [46]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6990
           1       1.00      1.00      1.00      6480

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470



In [47]:
rfc = RandomForestClassifier(random_state=0)
rfc.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [48]:
predictrfc = rfc.predict(xv_test)
rfc.score(xv_test, y_test)


0.9862657757980697

In [49]:
print(classification_report(y_test, predictrfc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      6990
           1       0.99      0.99      0.99      6480

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [54]:
gbc = GradientBoostingClassifier(random_state=0)
gbc.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [55]:
predictgbc = gbc.predict(xv_test)
gbc.score(xv_test, y_test)

0.9951744617668894

In [56]:
print(classification_report(y_test, predictgbc))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      6990
           1       0.99      1.00      0.99      6480

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470



In [57]:
def output_label(n):
  if n==0:
     return "Its a fake news"
  elif n==1:
     return "It is a genuine news"


In [58]:
## Manual testing
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DTC.predict(new_xv_test)
    pred_RFC = rfc.predict(new_xv_test)
    pred_GBC = gbc.predict(new_xv_test)
    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nRFC Prediction: {} \nGBC Prediction: {}".format(
        output_label(pred_LR[0]), output_label(pred_DT[0]), output_label(pred_RFC[0]), output_label(pred_GBC[0])))

In [ ]:
news_article = str(input())

In [ ]:
manual_testing(news_article)